# <font color='8EC044'> **Entrega 2 - Proyecto Kaggle**

### <font color='8EC044'> **Estudiante:** <font color='white'> Iván Steven Cuero Martínez

### <font color='8EC044'> **Programa:** <font color='white'> Ingeniería Industrial

## <font color='FF7F50'> **Descripción**

Las Pruebas Saber Pro son exámenes estandarizados que se administran en Colombia para evaluar la calidad y el nivel de conocimiento y competencias de los estudiantes de educación superior, es decir, de instituciones de educación superior como universidades y tecnológicos. Estas pruebas son parte de los esfuerzos del Gobierno de Colombia para monitorear y mejorar la calidad de la educación superior en el país.

Estas Pruebas constan cinco componentes genéricos, Inglés, Lectura Crítica, Competencias Ciudadanas, Razonamiento Cuantitativo y Comunicación Escrita.

El trabajo será crear un modelo de clasificación que, para cada estudiante, prediga qué desempeño va a tener: bajo, medio-bajo, medio-alto o alto.

## <font color='FF7F50'> **1. Inicialización**

### <font color='46B8A9'> **1.1. Librerías**

In [1]:
import warnings
warnings.filterwarnings("ignore")

# Datos
import os
from google.colab import files
import pandas as pd
import numpy as np
from itertools import product

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Modelado
from sklearn.preprocessing import LabelEncoder, MinMaxScaler,StandardScaler

### <font color='46B8A9'> **1.2. Descarga y carga de datos desde Kaggle**


**Instrucciones:**

**1.** Crear un archivo kaggle.json con el token de autenticación (en Kaggle → hacer clic en el ícono de usuario en la esquina superior derecha → configuración → API → crear nuevo token).

**2.** Subir el archivo kaggle.json a este espacio de trabajo de Colab.

In [2]:
# Subir el archivo kaggle.json
uploaded = files.upload()

Saving kaggle.json to kaggle.json


**3.** Configurar la autenticación con Kaggle usando el archivo kaggle.json y descargar los archivos de la competencia udea-ai-4-eng-20251-pruebas-saber-pro-colombia en el directorio actual.

In [3]:
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.11GB/s]


**4.** Descomprimir e inspeccionar datos.

In [4]:
!unzip udea*.zip > /dev/null

In [5]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185250 test.csv
   692501  10666231 143732449 train.csv
  1286075  15528571 207634372 total


**5.** Cargar train.csv con pandas.

In [6]:
df = pd.read_csv("train.csv")

### <font color='46B8A9'> **1.3. Descripción general de los datos**

In [7]:
# Obtener el número de filas y columnas del DataFrame
rows, cols = df.shape
print(f'Hay {rows} filas y {cols} columnas en el dataset')

Hay 692500 filas y 21 columnas en el dataset


In [8]:
# Mostrar información general del DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 21 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              692500 non-null  int64  
 1   PERIODO                         692500 non-null  int64  
 2   ESTU_PRGM_ACADEMICO             692500 non-null  object 
 3   ESTU_PRGM_DEPARTAMENTO          692500 non-null  object 
 4   ESTU_VALORMATRICULAUNIVERSIDAD  686213 non-null  object 
 5   ESTU_HORASSEMANATRABAJA         661643 non-null  object 
 6   FAMI_ESTRATOVIVIENDA            660363 non-null  object 
 7   FAMI_TIENEINTERNET              665871 non-null  object 
 8   FAMI_EDUCACIONPADRE             669322 non-null  object 
 9   FAMI_TIENELAVADORA              652727 non-null  object 
 10  FAMI_TIENEAUTOMOVIL             648877 non-null  object 
 11  ESTU_PRIVADO_LIBERTAD           692500 non-null  object 
 12  ESTU_PAGOMATRICU

## <font color='FF7F50'> **2. Limpieza y preprocesado de datos**

### <font color='46B8A9'> **2.1. Identificación y eliminación de variables (columnas) iguales**

Se observa que en el conjunto de datos existen dos columnas con nombres muy similares: 'FAMI_TIENEINTERNET' y 'FAMI_TIENEINTERNET.1', lo cual sugiere que podrían contener la misma información. Para confirmar esto, se compararon ambas columnas utilizando el método .equals() de pandas, que verifica si todos los elementos de ambas columnas son idénticos. Si el resultado es True, se concluye que las dos columnas son duplicadas y, por lo tanto, es apropiado eliminar una de ellas para evitar redundancia y reducir la dimensionalidad del conjunto de datos.

In [9]:
df1 = df.copy()  # Crea una copia del dataframe original

In [10]:
# Verificar si las columnas 'FAMI_TIENEINTERNET' y 'FAMI_TIENEINTERNET.1' contienen exactamente la misma información
df1['FAMI_TIENEINTERNET'].equals(df1['FAMI_TIENEINTERNET.1'])

True

In [11]:
# Eliminar columna 'FAMI_TIENEINTERNET.1'
df1.drop('FAMI_TIENEINTERNET.1', axis=1, inplace=True)

### <font color='46B8A9'> **2.2. Tratamiento de datos nulos**

En la entrega 1 se determinó que varias columnas categóricas del conjunto de datos contenían una pequeña proporción de valores nulos, la mayoría por debajo del 5% del total de observaciones. Dado que este porcentaje es bajo, no representa un problema grave para el análisis. En lugar de eliminar filas o aplicar métodos más complejos de imputación, se opta por reemplazar los valores nulos en estas columnas por la moda, es decir, la categoría más frecuente dentro de cada variable. Esta estrategia permite conservar la totalidad de los datos y reduce el riesgo de introducir sesgos, ya que mantiene la coherencia con la distribución original de cada variable. Además, al tratarse de variables categóricas, esta imputación es adecuada porque preserva la interpretabilidad del conjunto de datos y la simplicidad del proceso de preprocesamiento.



In [12]:
# Calcular la cantidad de valores nulos por columna en el DataFrame
null_counts = df1.isnull().sum()

# Filtrar solo las columnas que tienen al menos un valor nulo
null_counts = null_counts[null_counts > 0]

# Mostrar el número de valores nulos por columna (solo las que tienen nulos)
null_counts

,0
ESTU_VALORMATRICULAUNIVERSIDAD,6287
ESTU_HORASSEMANATRABAJA,30857
FAMI_ESTRATOVIVIENDA,32137
FAMI_TIENEINTERNET,26629
FAMI_EDUCACIONPADRE,23178
FAMI_TIENELAVADORA,39773
FAMI_TIENEAUTOMOVIL,43623
ESTU_PAGOMATRICULAPROPIO,6498
FAMI_TIENECOMPUTADOR,38103
FAMI_EDUCACIONMADRE,23664


In [13]:
# Iterar sobre todas las columnas categóricas que tienen valores nulos
for column in null_counts.index:
    # Obtener la moda (valor más frecuente) de la columna
    mode_value = df1[column].mode()[0]
    # Reemplazar los valores nulos en la columna con la moda
    df1[column].fillna(mode_value, inplace=True)

# Calcular el total de valores nulos restantes en todo el DataFrame (debería dar 0 si se imputaron todos)
remaining_nulls = df1.isnull().sum().sum()
remaining_nulls

np.int64(0)

In [14]:
# Verificar presencia de nulos
df1.isnull().sum()

,0
ID,0
PERIODO,0
ESTU_PRGM_ACADEMICO,0
ESTU_PRGM_DEPARTAMENTO,0
ESTU_VALORMATRICULAUNIVERSIDAD,0
ESTU_HORASSEMANATRABAJA,0
FAMI_ESTRATOVIVIENDA,0
FAMI_TIENEINTERNET,0
FAMI_EDUCACIONPADRE,0
FAMI_TIENELAVADORA,0


### <font color='46B8A9'> **2.3. Variable ID**

In [15]:
# La siguiente línea modifica directamente el DataFrame df1, estableciendo la columna 'ID' como índice
df1.set_index('ID', inplace=True)

In [16]:
df1

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
ID,,,,,,,,,,,,,,,,,,,
904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25096,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,Si,Si,Secundaria (Bachillerato) incompleta,medio-alto,0.237,0.271,0.271,0.311
754213,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Primaria incompleta,Si,No,N,No,Si,Secundaria (Bachillerato) incompleta,bajo,0.314,0.240,0.278,0.260
504185,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,Si,Si,Secundaria (Bachillerato) incompleta,medio-bajo,0.286,0.240,0.314,0.287


### <font color='46B8A9'> **2.4. Variable PERIODO**

Se crea una nueva columna llamada 'AÑO' a partir de la variable 'PERIODO', debido a que esta última contiene múltiples categorías que representan periodos académicos específicos (por ejemplo, 20195 o 20212), algunas de las cuales tienen una cantidad de observaciones considerablemente mayor que otras. Esta disparidad en la distribución puede generar desequilibrios en el análisis y en la construcción de los distintos modelos que se van a implementar. Al agrupar los periodos por año, se reduce la complejidad categórica de la variable y se homogeniza la distribución de los datos.

In [17]:
df1.PERIODO.value_counts()

,count
PERIODO,
20195,180873
20203,171838
20212,171412
20183,164818
20194,1472
20213,1178
20202,490
20184,254
20196,165


In [18]:
''' Se crea una nueva columna llamada 'AÑO' debido a que algunas categorías dentro de la variable 'PERIODO' son muy grandes,
 mientras que otras son demasiado pequeñas. Al agrupar los periodos en años, se facilita la interpretación de los resultados
 y se capturan tendencias más generales a lo largo del tiempo, sin perder información relevante de los periodos individuales.'''

# Definir un diccionario para mapear los valores de 'PERIODO' a años
agrupacionPERIODO = {
    20195: 2019,
    20203: 2020,
    20212: 2021,
    20183: 2018,
    20194: 2019,
    20213: 2021,
    20202: 2020,
    20184: 2018,
    20196: 2019
}

# Aplicar el mapeo de 'PERIODO' a 'AÑO' usando el diccionario
df1['AÑO'] = df1['PERIODO'].map(agrupacionPERIODO)

In [19]:
df1.AÑO.value_counts()

,count
AÑO,
2019,182510
2021,172590
2020,172328
2018,165072


### <font color='46B8A9'> **2.5. Variable ESTU_PRGM_ACADEMICO**

Para simplificar el análisis y mejorar la interpretabilidad de los datos, se agrupan las numerosas categorías originales de programas académicos en grupos más generales y manejables. Esta agrupación se realiza mediante una función que clasifica cada programa según palabras clave presentes en su nombre, asegurando que carreras similares se unan bajo una misma categoría amplia. Esta estrategia reduce la alta dimensionalidad que implicaría trabajar con muchas categorías individuales.

In [20]:
def clasificar_carrera(carrera):
    carrera = carrera.upper()

    # Ingenierías
    if 'INGENIER' in carrera or 'INGENIER¿' in carrera or 'INGENIERÌ' in carrera:
        return 'Ingeniería'

    # Ciencias de la Salud
    salud = ['MEDICINA', 'ENFERMER', 'ODONTOLOG', 'FISIOTERAP', 'FARMACIA',
             'NUTRICI', 'TERAPIA', 'OPTOMETR', 'BACTERIOLOG', 'BIOANALISIS',
             'INSTRUMENTACION QUIRURGICA', 'FONOAUDIOLOG', 'GERONTOLOG']
    if any(palabra in carrera for palabra in salud):
        return 'Ciencias de la Salud'

    # Ciencias Sociales y Humanidades
    sociales = ['DERECHO', 'PSICOLOG', 'SOCIOLOG', 'TRABAJO SOCIAL', 'CIENCIA POLITICA',
                'ANTROPOLOG', 'HISTORIA', 'FILOSOF', 'COMUNICACION', 'PERIODISMO']
    if any(palabra in carrera for palabra in sociales):
        return 'Ciencias Sociales y Humanidades'

    # Educación
    if 'EDUCACION' in carrera or 'EDUCACI¿N' in carrera or 'PEDAGOG' in carrera or 'LICENCIATURA' in carrera:
        return 'Educación'

    # Administración y Negocios
    admin = ['ADMINISTRACION', 'ADMINISTRACI¿N', 'ADMINISTRACIÒN', 'NEGOCIOS',
             'FINANZAS', 'CONTADUR', 'ECONOM', 'EMPRESA', 'MERCADEO', 'MARKETING',
             'COMERCIO', 'RELACIONES INTERNACIONALES', 'PUBLICIDAD']
    if any(palabra in carrera for palabra in admin):
        return 'Administración y Negocios'

    # Ciencias Básicas
    basicas = ['BIOLOG', 'QUIMICA', 'FISICA', 'MATEMATIC', 'ESTADISTIC', 'GEOLOG',
               'MICROBIOLOG', 'ECOLOG', 'CIENCIA', 'BIOQUIM']
    if any(palabra in carrera for palabra in basicas):
        return 'Ciencias Básicas'

    # Arte y Diseño
    arte = ['ARTE', 'DISEÑO', 'DISE¿O', 'MUSICA', 'TEATRO', 'DANZA', 'CINE',
            'BELLAS ARTES', 'GASTRONOM', 'CULINARIA']
    if any(palabra in carrera for palabra in arte):
        return 'Arte y Diseño'

    # Arquitectura
    if 'ARQUITECTURA' in carrera or 'URBANISMO' in carrera:
        return 'Arquitectura y Urbanismo'

    # Agronomía y afines
    if 'AGRONOM' in carrera or 'ZOOTECN' in carrera or 'AGROPECUAR' in carrera or 'AGROINDUSTRIAL' in carrera:
        return 'Agronomía y Ciencias Agropecuarias'

    # Tecnología
    if 'TECNOLOG' in carrera or 'INFORMATICA' in carrera or 'SISTEMAS' in carrera:
        return 'Tecnología e Informática'

    # Si no coincide con ninguna categoría anterior
    return 'Otras'

# Aplicar la función a tu columna
df1['PRGM_ACADEMICO'] = df1['ESTU_PRGM_ACADEMICO'].apply(clasificar_carrera)

# Verificar las categorías creadas
df1.PRGM_ACADEMICO.value_counts()

,count
PRGM_ACADEMICO,
Administración y Negocios,198410
Ingeniería,148431
Ciencias Sociales y Humanidades,141503
Educación,64442
Ciencias de la Salud,57211
Otras,37031
Arte y Diseño,16030
Ciencias Básicas,14856
Arquitectura y Urbanismo,11746


### <font color='46B8A9'> **2.6. Variable ESTU_PRGM_DEPARTAMENTO**

Para facilitar el análisis y reducir la cardinalidad de la variable 'ESTU_PRGM_DEPARTAMENTO', se agrupan los departamentos en seis grandes regiones geográficas de Colombia: Andina, Caribe, Pacífica, Orinoquía, Amazónica e Insular.

In [21]:
df1['ESTU_PRGM_DEPARTAMENTO'].unique()  # Muestra los valores únicos presentes en la columna

array(['BOGOTÁ', 'ATLANTICO', 'SANTANDER', 'ANTIOQUIA', 'HUILA', 'SUCRE',
       'CAQUETA', 'CUNDINAMARCA', 'BOLIVAR', 'TOLIMA', 'VALLE', 'QUINDIO',
       'RISARALDA', 'CORDOBA', 'META', 'LA GUAJIRA', 'BOYACA', 'NARIÑO',
       'CAUCA', 'NORTE SANTANDER', 'CESAR', 'PUTUMAYO', 'CALDAS',
       'MAGDALENA', 'CHOCO', 'CASANARE', 'ARAUCA', 'GUAVIARE', 'AMAZONAS',
       'VAUPES', 'SAN ANDRES'], dtype=object)

In [22]:
'''Se crea un diccionario de regiones porque agrupar los departamentos en categorías geográficas más amplias simplifica
   el análisis, reduce la cardinalidad de la variable y permite identificar patrones regionales. A partir de este diccionario,
   se crea una nueva columna llamada 'REGION' en el DataFrame para incorporar esta agrupación'''

# Crear un diccionario de regiones
departamentos_regiones = {
    # Región Andina
    'BOGOTÁ': 'Andina',
    'SANTANDER': 'Andina',
    'ANTIOQUIA': 'Andina',
    'HUILA': 'Andina',
    'CUNDINAMARCA': 'Andina',
    'TOLIMA': 'Andina',
    'QUINDIO': 'Andina',
    'RISARALDA': 'Andina',
    'BOYACA': 'Andina',
    'NORTE SANTANDER': 'Andina',
    'CALDAS': 'Andina',

    # Región Amazónica
    'CAQUETA': 'Amazónica',
    'PUTUMAYO': 'Amazónica',
    'GUAVIARE': 'Amazónica',
    'AMAZONAS': 'Amazónica',
    'VAUPES': 'Amazónica',

    # Región Pacífica
    'VALLE': 'Pacífica',
    'NARIÑO': 'Pacífica',
    'CAUCA': 'Pacífica',
    'CHOCO': 'Pacífica',

    # Región Caribe
    'ATLANTICO': 'Caribe',
    'SUCRE': 'Caribe',
    'BOLIVAR': 'Caribe',
    'CORDOBA': 'Caribe',
    'LA GUAJIRA': 'Caribe',
    'CESAR': 'Caribe',
    'MAGDALENA': 'Caribe',

    # Región Orinoquía
    'META': 'Orinoquía',
    'CASANARE': 'Orinoquía',
    'ARAUCA': 'Orinoquía',

    # Región Insular
    'SAN ANDRES': 'Insular'
}

# Se agrupan los departamentos en las siguientes regiones geográficas de Colombia:
# Andina, Amazónica, Pacífica, Caribe, Orinoquía e Insular y se agrega la variable REGION a df1

df1['REGION'] = df1['ESTU_PRGM_DEPARTAMENTO'].map(departamentos_regiones)

In [23]:
df1.REGION.value_counts()

,count
REGION,
Andina,499712
Caribe,105080
Pacífica,73802
Orinoquía,10351
Amazónica,3545
Insular,10


### <font color='46B8A9'> **2.7. Variable ESTU_VALORMATRICULAUNIVERSIDAD**

La variable 'ESTU_VALORMATRICULAUNIVERSIDAD' contiene categorías en forma de rangos de valores monetarios. Para facilitar su uso en modelos de machine learning, se transforma en una variable numérica asignando a cada rango un valor representativo basado en el promedio del intervalo correspondiente. Esta transformación conserva el orden y la magnitud relativa de los valores, lo cual resulta útil para algoritmos que capturan relaciones numéricas y ordinales, como regresión logística, árboles de decisión y modelos de ensamble. Además, al convertir esta variable en formato numérico se evita la creación de múltiples columnas mediante codificación one-hot, lo que reduce la dimensionalidad del dataset. Esta conversión no solo optimiza el procesamiento, sino que también puede mejorar la precisión del modelo si existe una relación significativa entre el valor de la matrícula y el rendimiento global del estudiante.

In [24]:
df1.ESTU_VALORMATRICULAUNIVERSIDAD.value_counts()

,count
ESTU_VALORMATRICULAUNIVERSIDAD,
Entre 1 millón y menos de 2.5 millones,210335
Entre 2.5 millones y menos de 4 millones,127430
Menos de 500 mil,80263
Entre 500 mil y menos de 1 millón,78704
Entre 4 millones y menos de 5.5 millones,69736
Más de 7 millones,68014
Entre 5.5 millones y menos de 7 millones,38490
No pagó matrícula,19528


In [25]:
# Se asigna el valor promedio de pago de matrícula a cada categoría dentro de la variable
valormat = {'Entre 1 millón y menos de 2.5 millones': 1.75,
    'Entre 2.5 millones y menos de 4 millones': 3.25,
    'Menos de 500 mil': .250,
    'Entre 500 mil y menos de 1 millón': .75,
    'Entre 4 millones y menos de 5.5 millones': 4.75,
    'Más de 7 millones': 7.75,
    'Entre 5.5 millones y menos de 7 millones': 6.25,
    'No pagó matrícula': 0}

# Usar map para transformar los valores
df1['ESTU_VALORMATRICULAUNIVERSIDAD'] = df1['ESTU_VALORMATRICULAUNIVERSIDAD'].map(valormat)

# Contar los valores únicos
df1.ESTU_VALORMATRICULAUNIVERSIDAD.value_counts()

,count
ESTU_VALORMATRICULAUNIVERSIDAD,
1.75,210335
3.25,127430
0.25,80263
0.75,78704
4.75,69736
7.75,68014
6.25,38490
0.00,19528


### <font color='46B8A9'> **2.8. Variable ESTU_HORASSEMANATRABAJA**

Con el fin de convertir la variable ESTU_HORASSEMANATRABAJA en un formato numérico útil para el análisis estadístico y la implementación de modelos de aprendizaje supervisado, se asigna a cada categoría un valor promedio representativo de horas trabajadas por semana. Esta transformación facilita el tratamiento cuantitativo de la variable, mejora la interpretabilidad de los datos y permite que los modelos capturen de forma más precisa la posible relación entre el número de horas trabajadas y la variable respuesta (rendimiento global).

In [26]:
df1.ESTU_HORASSEMANATRABAJA.value_counts()

,count
ESTU_HORASSEMANATRABAJA,
Más de 30 horas,280209
0,116550
Entre 11 y 20 horas,115857
Entre 21 y 30 horas,92693
Menos de 10 horas,87191


In [27]:
# Se asigna el valor promedio de horas a cada categoría dentro de la variable
horasem = {'0': 0,
    'Menos de 10 horas': 5,
    'Entre 11 y 20 horas': 15.5,
    'Entre 21 y 30 horas': 25.5,
    'Más de 30 horas': 35.5}

# Usar map para transformar los valores
df1['ESTU_HORASSEMANATRABAJA'] = df1['ESTU_HORASSEMANATRABAJA'].map(horasem)

# Contar los valores únicos
df1['ESTU_HORASSEMANATRABAJA'].value_counts()

,count
ESTU_HORASSEMANATRABAJA,
35.5,280209
0.0,116550
15.5,115857
25.5,92693
5.0,87191


### <font color='46B8A9'> **2.9. Variable FAMI_ESTRATOVIVIENDA**

Se transforma la variable 'FAMI_ESTRATOVIVIENDA' para mejorar su utilidad en los modelos de aprendizaje supervisado. Se reemplazan los nombres de los estratos por sus respectivos valores numéricos, preservando así su naturaleza ordinal (es decir, una jerarquía de niveles socioeconómicos). Esta conversión permite que los modelos interpreten correctamente la relación de orden entre los estratos y facilita el procesamiento matemático de la variable.

In [28]:
df1.FAMI_ESTRATOVIVIENDA.value_counts()

,count
FAMI_ESTRATOVIVIENDA,
Estrato 2,264808
Estrato 3,210685
Estrato 1,111991
Estrato 4,65514
Estrato 5,23608
Estrato 6,12605
Sin Estrato,3289


In [29]:
# Se reemplazan los nombres de los estratos por sus valores numéricos, manteniendo su naturaleza ordinal.
df1['FAMI_ESTRATOVIVIENDA'] = df1['FAMI_ESTRATOVIVIENDA'].replace({
    'Estrato 6': 6,
    'Estrato 1': 1,
    'Estrato 2': 2,
    'Estrato 3': 3,
    'Estrato 4': 4,
    'Estrato 5': 5
    })

df1.FAMI_ESTRATOVIVIENDA.value_counts()

,count
FAMI_ESTRATOVIVIENDA,
2,264808
3,210685
1,111991
4,65514
5,23608
6,12605
Sin Estrato,3289


### <font color='46B8A9'> **2.10. Variables FAMI_EDUCACIONPADRE y FAMI_EDUCACIONMADRE**

Se unifican los valores inciertos de las variables 'FAMI_EDUCACIONPADRE' y 'FAMI_EDUCACIONMADRE' al reemplazar las categorías "No sabe" y "No Aplica" por una única categoría llamada "Indeterminado". Esta transformación se realiza para simplificar el conjunto de datos, reducir la cardinalidad de las variables y evitar la dispersión de información faltante en múltiples etiquetas que representan esencialmente la misma condición de desconocimiento. Con ello, se mejora la calidad de los datos y se facilita la interpretación por parte de los modelos de aprendizaje supervisado.

In [30]:
df1.FAMI_EDUCACIONPADRE.value_counts()

,count
FAMI_EDUCACIONPADRE,
Secundaria (Bachillerato) completa,151467
Primaria incompleta,125675
Educación profesional completa,83117
Secundaria (Bachillerato) incompleta,71654
Técnica o tecnológica completa,62995
Primaria completa,55958
Postgrado,44169
Educación profesional incompleta,27084
Técnica o tecnológica incompleta,22552


In [31]:
df1.FAMI_EDUCACIONMADRE.value_counts()

,count
FAMI_EDUCACIONMADRE,
Secundaria (Bachillerato) completa,165408
Primaria incompleta,99420
Técnica o tecnológica completa,89542
Educación profesional completa,85326
Secundaria (Bachillerato) incompleta,81012
Primaria completa,56125
Postgrado,46246
Técnica o tecnológica incompleta,27533
Educación profesional incompleta,22470


In [32]:
''' Se reemplazan las categorías 'No sabe' y 'No Aplica' por 'Indeterminado' en las columnas de educación de madre y padre,
    para unificar los valores inciertos.'''

# Reemplazar valores inciertos en la educación de la madre
df1['FAMI_EDUCACIONMADRE'] = [
    'Indeterminado' if i in ['No sabe', 'No Aplica'] else i
    for i in df1['FAMI_EDUCACIONMADRE'].values
]

# Reemplazar valores inciertos en la educación del padre
df1['FAMI_EDUCACIONPADRE'] = [
    'Indeterminado' if i in ['No sabe', 'No Aplica'] else i
    for i in df1['FAMI_EDUCACIONPADRE'].values
]

In [33]:
df1.FAMI_EDUCACIONPADRE.value_counts()

,count
FAMI_EDUCACIONPADRE,
Secundaria (Bachillerato) completa,151467
Primaria incompleta,125675
Educación profesional completa,83117
Secundaria (Bachillerato) incompleta,71654
Técnica o tecnológica completa,62995
Primaria completa,55958
Postgrado,44169
Educación profesional incompleta,27084
Indeterminado,25821


In [34]:
df1.FAMI_EDUCACIONMADRE.value_counts()

,count
FAMI_EDUCACIONMADRE,
Secundaria (Bachillerato) completa,165408
Primaria incompleta,99420
Técnica o tecnológica completa,89542
Educación profesional completa,85326
Secundaria (Bachillerato) incompleta,81012
Primaria completa,56125
Postgrado,46246
Técnica o tecnológica incompleta,27533
Educación profesional incompleta,22470


## <font color='FF7F50'> **3. Selección de variables y preparación de los datos**

### <font color='46B8A9'> **3.1. Selección de variables**

In [35]:
# Mostrar información general del DataFrame
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 692500 entries, 904256 to 933374
Data columns (total 22 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   PERIODO                         692500 non-null  int64  
 1   ESTU_PRGM_ACADEMICO             692500 non-null  object 
 2   ESTU_PRGM_DEPARTAMENTO          692500 non-null  object 
 3   ESTU_VALORMATRICULAUNIVERSIDAD  692500 non-null  float64
 4   ESTU_HORASSEMANATRABAJA         692500 non-null  float64
 5   FAMI_ESTRATOVIVIENDA            692500 non-null  object 
 6   FAMI_TIENEINTERNET              692500 non-null  object 
 7   FAMI_EDUCACIONPADRE             692500 non-null  object 
 8   FAMI_TIENELAVADORA              692500 non-null  object 
 9   FAMI_TIENEAUTOMOVIL             692500 non-null  object 
 10  ESTU_PRIVADO_LIBERTAD           692500 non-null  object 
 11  ESTU_PAGOMATRICULAPROPIO        692500 non-null  object 
 12  FAMI_TIENECOMPUT

<font color='white'> **Análisis de correlación entre variables numéricas y variable respuesta:**

Para evaluar la relación entre las variables predictoras ('ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'coef_1', 'coef_2', 'coef_3', 'coef_4', 'AÑO') y la variable respuesta ('RENDIMIENTO_GLOBAL'), se utilizó el coeficiente de correlación tipo Eta (η), una medida adecuada cuando la variable dependiente es categórica y las independientes son numéricas. Este coeficiente permite cuantificar la fuerza de asociación entre una variable categórica y una continua, arrojando valores entre 0 (sin asociación) y 1 (asociación perfecta). Aplicar esta técnica permite identificar qué variables tienen mayor capacidad explicativa sobre el rendimiento global, lo que es fundamental para la selección de variables relevantes en modelos de aprendizaje supervisado.

In [36]:
# 1. Función para calcular el coeficiente de correlación tipo Eta
def correlation_ratio(categories, measurements):
    """
    categories: array-like categórico
    measurements: array-like numérico
    devuelve: eta, valor entre 0 y 1 que mide la asociación
    """
    # Convertimos categorías a enteros 0,1,2,...
    fcat, _ = pd.factorize(categories)
    cats = np.unique(fcat)
    mean_total = np.nanmean(measurements)

    # Suma de cuadrados entre categorías
    ss_between = sum(
        len(measurements[fcat == cat]) *
        (np.nanmean(measurements[fcat == cat]) - mean_total)**2
        for cat in cats
    )
    # Suma total de cuadrados
    ss_total = np.nansum((measurements - mean_total)**2)

    # Eta = sqrt(SS_between / SS_total)
    return np.sqrt(ss_between / ss_total) if ss_total > 0 else np.nan

# 2. Cálculo de η para cada coeficiente
eta_values = {}
for col in ['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'coef_1','coef_2','coef_3','coef_4', 'AÑO']:
    eta = correlation_ratio(df1['RENDIMIENTO_GLOBAL'], df1[col].values)
    eta_values[col] = eta

# 3. Mostrar resultados ordenados
eta_df = pd.DataFrame.from_dict(eta_values, orient='index', columns=['eta']) \
            .sort_values(by='eta', ascending=False)
eta_df

,eta
ESTU_VALORMATRICULAUNIVERSIDAD,0.269710
coef_1,0.252061
coef_2,0.195314
coef_4,0.135324
ESTU_HORASSEMANATRABAJA,0.131308
coef_3,0.060940
AÑO,0.059160


En este caso, la variable ESTU_VALORMATRICULAUNIVERSIDAD mostró la mayor correlación, seguida de coef_1, coef_2, coef_4 y ESTU_HORASSEMANATRABAJA, lo cual sugiere que estas variables podrían tener un peso predictivo relevante.

<font color='white'> **Análisis de correlación entre variables categóricas y variable respuesta:**

Para identificar qué variables categóricas tienen una relación estadísticamente significativa con la variable de respuesta 'RENDIMIENTO_GLOBAL', se utiliza la prueba de Chi-cuadrado de independencia, que permite evaluar si existe una asociación entre dos variables categóricas. Al calcular el valor del estadístico Chi² y su correspondiente p-valor para cada variable, se puede determinar qué características del conjunto de datos están significativamente relacionadas con el rendimiento académico de los estudiantes. Este análisis es fundamental en la etapa de selección de variables, ya que permite reducir la dimensionalidad de los modelos, eliminando variables irrelevantes y conservando aquellas que aportan información útil para la predicción.

In [37]:
from scipy.stats import chi2_contingency

chi2_results = []

# Seleccionar solo las columnas categóricas, excluyendo la variable objetivo
categorical_columns = df1.columns[df1.dtypes == 'object'].tolist()
categorical_columns.remove('RENDIMIENTO_GLOBAL')

# Calcular la prueba Chi-cuadrado para cada variable categórica respecto a la variable objetivo
for column in categorical_columns:
    contingency_table = pd.crosstab(df1[column], df1['RENDIMIENTO_GLOBAL'])  # Tabla de contingencia
    chi2, p_value, _, _ = chi2_contingency(contingency_table)  # Prueba Chi-cuadrado
    chi2_results.append((column, chi2, p_value))  # Guarda resultados

# Convertir los resultados en un DataFrame y ordenar por significancia estadística (p-value)
chi2_results_df = pd.DataFrame(chi2_results, columns=['Variable', 'Chi2', 'p-value'])
chi2_results_df.sort_values(by='p-value', inplace=True)

chi2_results_df

,Variable,Chi2,p-value
0,ESTU_PRGM_ACADEMICO,143741.141869,0.000000
1,ESTU_PRGM_DEPARTAMENTO,27731.258060,0.000000
2,FAMI_ESTRATOVIVIENDA,54872.985913,0.000000
3,FAMI_TIENEINTERNET,14122.196556,0.000000
4,FAMI_EDUCACIONPADRE,57680.731917,0.000000
5,FAMI_TIENELAVADORA,7823.648464,0.000000
6,FAMI_TIENEAUTOMOVIL,19588.204011,0.000000
8,ESTU_PAGOMATRICULAPROPIO,28125.037722,0.000000
12,REGION,8463.446239,0.000000
9,FAMI_TIENECOMPUTADOR,12320.133185,0.000000


A partir de los resultados de la prueba de Chi-cuadrado, podemos interpretar la relación entre cada variable categórica y la variable de respuesta 'RENDIMIENTO_GLOBAL' evaluando el p-valor asociado a cada una.

**Interpretación de los resultados:**

* p-valor ≤ 0.05: La variable tiene una asociación estadísticamente significativa con RENDIMIENTO_GLOBAL. Es decir, es relevante para el análisis y podría contribuir al modelo predictivo.

* p-valor > 0.05: No hay evidencia suficiente para afirmar que existe una relación entre la variable y RENDIMIENTO_GLOBAL, por lo tanto, puede considerarse irrelevante para efectos de predicción.

'ESTU_PRIVADO_LIBERTAD' tiene un p-valor = 0.269961, lo cual indica que no hay evidencia estadística suficiente para afirmar que esta variable esté asociada al rendimiento académico. Por tanto, puede descartarse del análisis o de los modelos predictivos.

In [38]:
df1.columns

Index(['PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD',
       'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR',
       'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL', 'coef_1', 'coef_2',
       'coef_3', 'coef_4', 'AÑO', 'PRGM_ACADEMICO', 'REGION'],
      dtype='object')

In [39]:
# Se elimina la variable respuesta y seis variables que no se usarán en el modelo
X_features = df1.drop(['RENDIMIENTO_GLOBAL', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_PRIVADO_LIBERTAD',
                       'coef_3', 'AÑO'], axis=1, inplace=False)

In [40]:
X_features.dtypes

,0
ESTU_VALORMATRICULAUNIVERSIDAD,float64
ESTU_HORASSEMANATRABAJA,float64
FAMI_ESTRATOVIVIENDA,object
FAMI_TIENEINTERNET,object
FAMI_EDUCACIONPADRE,object
FAMI_TIENELAVADORA,object
FAMI_TIENEAUTOMOVIL,object
ESTU_PAGOMATRICULAPROPIO,object
FAMI_TIENECOMPUTADOR,object
FAMI_EDUCACIONMADRE,object


### <font color='46B8A9'> **3.2. Transformación de variables categóricas**

In [41]:
# Se identifican las columnas que son binarias (es decir, que tienen exactamente dos categorías distintas)
columnas_binarias = [col for col in X_features.columns if X_features[col].nunique() == 2]
columnas_binarias

['FAMI_TIENEINTERNET',
 'FAMI_TIENELAVADORA',
 'FAMI_TIENEAUTOMOVIL',
 'ESTU_PAGOMATRICULAPROPIO',
 'FAMI_TIENECOMPUTADOR']

<font color='white'> **Codificación One-Hot simplificada para variables categóricas binarias**:

In [42]:
# Aplicar codificación One-Hot simplificada (drop_first=True) a variables categóricas binarias
X_features = pd.get_dummies(X_features, columns=columnas_binarias,
                            drop_first=True  # Evita la multicolinealidad generando solo una columna por variable (1 = 'Sí', 0 = 'No')
)
X_features.head()

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,coef_1,coef_2,coef_4,PRGM_ACADEMICO,REGION,FAMI_TIENEINTERNET_Si,FAMI_TIENELAVADORA_Si,FAMI_TIENEAUTOMOVIL_Si,ESTU_PAGOMATRICULAPROPIO_Si,FAMI_TIENECOMPUTADOR_Si
ID,,,,,,,,,,,,,,,
904256,6.25,5.0,3,Técnica o tecnológica incompleta,Postgrado,0.322,0.208,0.267,Ciencias de la Salud,Andina,True,True,True,False,True
645256,3.25,0.0,3,Técnica o tecnológica completa,Técnica o tecnológica incompleta,0.311,0.215,0.264,Ciencias Sociales y Humanidades,Caribe,False,True,False,False,True
308367,3.25,35.5,3,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,0.297,0.214,0.264,Administración y Negocios,Andina,True,True,False,False,False
470353,4.75,0.0,4,Indeterminado,Secundaria (Bachillerato) completa,0.485,0.172,0.190,Administración y Negocios,Andina,True,True,False,False,True
989032,3.25,25.5,3,Primaria completa,Primaria completa,0.316,0.232,0.294,Ciencias Sociales y Humanidades,Andina,True,True,True,False,True


<font color='white'> **Codificación One-Hot Encoding para variables categóricas multiclase:**

In [43]:
# Aplicar One-Hot Encoding (para variables con 3 o más categorías)
# Lista de variables categóricas a transformar
ohe_vars = ['FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE', 'PRGM_ACADEMICO', 'REGION']

# Generar las variables dummies para las columnas seleccionadas
X_features = pd.get_dummies(X_features, columns=ohe_vars)
X_features.head()

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,coef_1,coef_2,coef_4,FAMI_TIENEINTERNET_Si,FAMI_TIENELAVADORA_Si,FAMI_TIENEAUTOMOVIL_Si,ESTU_PAGOMATRICULAPROPIO_Si,FAMI_TIENECOMPUTADOR_Si,...,PRGM_ACADEMICO_Educación,PRGM_ACADEMICO_Ingeniería,PRGM_ACADEMICO_Otras,PRGM_ACADEMICO_Tecnología e Informática,REGION_Amazónica,REGION_Andina,REGION_Caribe,REGION_Insular,REGION_Orinoquía,REGION_Pacífica
ID,,,,,,,,,,,,,,,,,,,,,
904256,6.25,5.0,0.322,0.208,0.267,True,True,True,False,True,...,False,False,False,False,False,True,False,False,False,False
645256,3.25,0.0,0.311,0.215,0.264,False,True,False,False,True,...,False,False,False,False,False,False,True,False,False,False
308367,3.25,35.5,0.297,0.214,0.264,True,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
470353,4.75,0.0,0.485,0.172,0.190,True,True,False,False,True,...,False,False,False,False,False,True,False,False,False,False
989032,3.25,25.5,0.316,0.232,0.294,True,True,True,False,True,...,False,False,False,False,False,True,False,False,False,False


Aunque muchos algoritmos de machine learning pueden procesar valores booleanos, convertirlos explícitamente a enteros (0 y 1) evita ambigüedades, garantiza compatibilidad con todas las librerías de modelado y facilita el análisis y visualización de los datos. Esta conversión también ayuda a mantener consistencia en el tipo de datos cuando se combinan con otras variables ya numéricas.

In [44]:
# Convertir columnas booleanas a enteros explícitamente
for col in X_features.columns:
    if X_features[col].dtype == bool:
        X_features[col] = X_features[col].astype(int)

X_features.head()

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,coef_1,coef_2,coef_4,FAMI_TIENEINTERNET_Si,FAMI_TIENELAVADORA_Si,FAMI_TIENEAUTOMOVIL_Si,ESTU_PAGOMATRICULAPROPIO_Si,FAMI_TIENECOMPUTADOR_Si,...,PRGM_ACADEMICO_Educación,PRGM_ACADEMICO_Ingeniería,PRGM_ACADEMICO_Otras,PRGM_ACADEMICO_Tecnología e Informática,REGION_Amazónica,REGION_Andina,REGION_Caribe,REGION_Insular,REGION_Orinoquía,REGION_Pacífica
ID,,,,,,,,,,,,,,,,,,,,,
904256,6.25,5.0,0.322,0.208,0.267,1,1,1,0,1,...,0,0,0,0,0,1,0,0,0,0
645256,3.25,0.0,0.311,0.215,0.264,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0
308367,3.25,35.5,0.297,0.214,0.264,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
470353,4.75,0.0,0.485,0.172,0.190,1,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
989032,3.25,25.5,0.316,0.232,0.294,1,1,1,0,1,...,0,0,0,0,0,1,0,0,0,0


### <font color='46B8A9'> **3.3. Normalización de variables numéricas (Z-score)**

In [45]:
# Almacenar variables númericas
numcol = ['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'coef_1', 'coef_2', 'coef_4']

# Escalamiento con Z-Score
scaler = StandardScaler()
for col in numcol:
    X_features[[col]] = scaler.fit_transform(X_features[[col]])

X_features

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,coef_1,coef_2,coef_4,FAMI_TIENEINTERNET_Si,FAMI_TIENELAVADORA_Si,FAMI_TIENEAUTOMOVIL_Si,ESTU_PAGOMATRICULAPROPIO_Si,FAMI_TIENECOMPUTADOR_Si,...,PRGM_ACADEMICO_Educación,PRGM_ACADEMICO_Ingeniería,PRGM_ACADEMICO_Otras,PRGM_ACADEMICO_Tecnología e Informática,REGION_Amazónica,REGION_Andina,REGION_Caribe,REGION_Insular,REGION_Orinoquía,REGION_Pacífica
ID,,,,,,,,,,,,,,,,,,,,,
904256,1.488838,-1.133390,0.437002,-0.556223,0.060296,1,1,1,0,1,...,0,0,0,0,0,1,0,0,0,0
645256,0.182581,-1.487564,0.346934,-0.481341,0.016142,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0
308367,0.182581,1.027071,0.232301,-0.492038,0.016142,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
470353,0.835710,-1.487564,1.771650,-0.941332,-1.072993,1,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
989032,0.182581,0.318723,0.387874,-0.299484,0.457683,1,1,1,0,1,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25096,-0.905967,-0.389625,-0.258980,0.117717,0.707890,1,1,0,1,1,...,0,0,0,0,0,0,1,0,0,0
754213,0.182581,1.027071,0.371498,-0.213904,-0.042730,1,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
504185,-0.470548,-1.133390,0.142233,-0.213904,0.354657,1,1,0,1,1,...,0,0,1,0,0,1,0,0,0,0


### <font color='46B8A9'> **3.4. Codificación y distribución de la variable objetivo**

Antes de implementar cualquier modelo predictivo, es necesario preparar adecuadamente la variable objetivo 'RENDIMIENTO_GLOBAL'. Esta columna contiene etiquetas categóricas que representan niveles de desempeño académico (como 'bajo', 'medio-bajo', 'medio-alto' y 'alto'). Por ello, se procede a convertir dichas categorías en valores numéricos discretos mediante un mapeo explícito que preserve su orden jerárquico. Esta transformación permite que los modelos interpreten correctamente la estructura ordinal de la variable, lo que facilita tanto el entrenamiento como la evaluación del desempeño del clasificador.

In [46]:
df1.RENDIMIENTO_GLOBAL.value_counts()

,count
RENDIMIENTO_GLOBAL,
alto,175619
bajo,172987
medio-bajo,172275
medio-alto,171619


In [47]:
# Separar la variable respuesta (dependiente) y luego convertirla en valores discretos para la predicción
y_target = df1['RENDIMIENTO_GLOBAL']  # Variable a predecir

# Diccionario para transformar los valores categóricos en numéricos
y_target_map = {'alto': 3, 'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2}

# Aplicar la transformación directamente a la Serie y_target
y_target = y_target.map(y_target_map)

# Verificar distribución de clases
y_target.value_counts()

,count
RENDIMIENTO_GLOBAL,
3,175619
0,172987
1,172275
2,171619


In [48]:
# Gráfico variable respuesta a partir de y_target
counts = y_target.value_counts().reset_index()
counts.columns = ['RENDIMIENTO_GLOBAL', 'Count']  # Renombrar las columnas

# Calcular porcentaje y etiqueta
counts['Percentage'] = (counts['Count'] / counts['Count'].sum()) * 100
counts['Label'] = counts['Count'].astype(str) + ' (' + counts['Percentage'].round(1).astype(str) + '%)'

# Crear gráfico
fig = px.bar(
    counts,
    x='RENDIMIENTO_GLOBAL',
    y='Count',
    color='RENDIMIENTO_GLOBAL',
    title='Distribución de RENDIMIENTO_GLOBAL',
    labels={'RENDIMIENTO_GLOBAL': 'Rendimiento Global', 'Count': 'Frecuencia'},
    text='Label',
    width=1000,
    height=600
)

fig.show()

In [49]:
counts

,RENDIMIENTO_GLOBAL,Count,Percentage,Label
0,3,175619,25.360144,175619 (25.4%)
1,0,172987,24.980072,172987 (25.0%)
2,1,172275,24.877256,172275 (24.9%)
3,2,171619,24.782527,171619 (24.8%)


In [50]:
y_target

,RENDIMIENTO_GLOBAL
ID,
904256,2
645256,0
308367,0
470353,3
989032,1
...,...
25096,2
754213,0
504185,1
